# Capstone Project: Segmenting and Clustering Neighborhoods in Toronto

## Part 3 - Data Analysis

**NOTE**
I will be using only the central Toronto boroughs for my analysis

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

In [2]:
CLIENT_ID = 'SN0VIJAY21QJWFCYLHZ40KWI3KNJGBQGKYKMWALYK4UFXM2C' # your Foursquare ID
CLIENT_SECRET = 'R4O5CJFTGFYSOXFXIUYMZODZLJLPKDQWGLKUL2EY4XA3P40I' # your Foursquare Secret
VERSION = '20180604'

In [3]:
df_T = pd.read_csv('TNC_myLatLon.csv.csv').drop('Unnamed: 0', axis=1)
df_T = df_T['Bourough'].str.contains("Toronto")

#reorg_cols = [df_T.columns[1]] + [df_T.columns[0]] +list(df_T.columns[2:5])
#df_T = df_T[reorg_cols]
df_T.head()

FileNotFoundError: [Errno 2] File TNC_myLatLon.csv.csv does not exist: 'TNC_myLatLon.csv.csv'

In [17]:
map_toronto= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postal_code, neighbourhood in zip(df_T['Latitude'], df_T['Longitude'], df_T['Postal Code'], df_T['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, postal_code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 

map_toronto